<a href="https://colab.research.google.com/github/NHagar/cc-genealogy/blob/main/analysis/check_datasets_for_urls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install
Because colab still doesn't have datasets by default smh

In [1]:
!pip install datasets

# Check for URLs in the dataset

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [3]:
ds_df = pd.read_csv('https://raw.githubusercontent.com/NHagar/cc-genealogy/refs/heads/main/hf_dataset_registry/dataset_pool_text_1m%2B.csv')

In [4]:
ds_df_sample = ds_df.sample(100)
ds_df_sample

,id,hf_url
653,alexantonov/chuvash_mono,https://huggingface.co/datasets/alexantonov/chuvash_mono
12406,NeutrinoPit/OpenSubtitles2024-en-ar-batch49,https://huggingface.co/datasets/NeutrinoPit/OpenSubtitles2024-en-ar-batch49
6715,yiyic/oscar_es_train,https://huggingface.co/datasets/yiyic/oscar_es_train
7901,gowitheflow/supervised-multilingual,https://huggingface.co/datasets/gowitheflow/supervised-multilingual
1499,Babelscape/multinerd,https://huggingface.co/datasets/Babelscape/multinerd
...,...,...
6411,mteb/eurlex-multilingual,https://huggingface.co/datasets/mteb/eurlex-multilingual
1437,FremyCompany/BioLORD-Dataset,https://huggingface.co/datasets/FremyCompany/BioLORD-Dataset
9164,KathirKs/CC-MAIN-2017-13_row_wise_20240926_154028,https://huggingface.co/datasets/KathirKs/CC-MAIN-2017-13_row_wise_20240926_154028
10101,1231czx/stage2_ep1_6_first8,https://huggingface.co/datasets/1231czx/stage2_ep1_6_first8


In [5]:
from datasets import load_dataset, get_dataset_split_names, get_dataset_config_names
import itertools
import logging
import re

logging.getLogger("datasets").setLevel(logging.ERROR)

# URL-related terms
url_related_terms = [
    'url', 'link', 'domain', 'website', 'site', 'web',
    'http', 'https', 'www', 'uri', 'href', 'address', 'hyperlink',
    'path', 'redirect', 'source', 'anchor', 'api'
]

url_regex = re.compile(r'https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+|(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?://(?:www\.|(?!www))[a-zA-Z0-9]+\.[^\s]{2,}|www\.[a-zA-Z0-9]+\.[^\s]{2,}')

# regex pattern for faster matching
url_pattern = re.compile('|'.join(url_related_terms), re.IGNORECASE)
global total_errors
total_errors = 0

def check_for_urls_in_dataset(dataset_id):
    try:
        # check if the dataset requires a config
        config_name = None

        try:
            available_configs = get_dataset_config_names(dataset_id, trust_remote_code=True)
            if available_configs:
                # first available config as default
                config_name = available_configs[0]
                print(f"Using config '{config_name}' for dataset {dataset_id}")
        except Exception as config_error:
            print(f"Warning when checking configs for {dataset_id}: {config_error}")

        # check available splits - include config when checking splits too
        try:
            if config_name:
                # For datasets with configs, provide the config name
                available_splits = get_dataset_split_names(dataset_id, config_name=config_name, trust_remote_code=True)
            else:
                available_splits = get_dataset_split_names(dataset_id, trust_remote_code=True)

            split_to_use = 'train' if 'train' in available_splits else available_splits[0]
        except Exception as split_error:
            # default to 'train'
            split_to_use = 'train'
            print(f"Warning: Could not get splits for {dataset_id}: {split_error}")

        # When loading the dataset, include the config if needed
        if config_name:
            dataset = load_dataset(dataset_id, config_name, split=split_to_use,
                                  streaming=True, trust_remote_code=True)
        else:
            dataset = load_dataset(dataset_id, split=split_to_use,
                                  streaming=True, trust_remote_code=True)

        # check access to features
        if dataset.features is None:
            print(f"Warning: No features found for {dataset_id}")
            return False

        column_names = dataset.features.keys()

        # check if any column names suggest URL content
        url_related_columns = [column for column in column_names if url_pattern.search(column)]

        if url_related_columns:
            sample_size = 10
            sample_dataset = dataset.take(sample_size)

            # check if any URL-related columns actually contain URLs
            for column in url_related_columns:
                for row in sample_dataset:
                    value = row.get(column, "")
                    # if value is a string and contains a URL
                    if isinstance(value, str) and url_regex.search(value):
                        return True
            return False

        # no URL-related column names were found
        return False

    except Exception as e:
        global total_errors
        total_errors += 1
        print(f"Error {total_errors} processing {dataset_id}: {e}")
        return False

In [6]:
import logging
from tqdm.auto import tqdm

# Apply with progress bar
tqdm.pandas(desc="Checking for URLs")
ds_df_sample['does_have_urls'] = ds_df_sample['id'].progress_apply(check_for_urls_in_dataset)

# reorder columns
cols = ['does_have_urls'] + [col for col in ds_df_sample.columns if col != 'does_have_urls']
ds_df_sample = ds_df_sample[cols]
# display
ds_df_sample

Checking for URLs:   0%|          | 0/100 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/490 [00:00<?, ?B/s]

Using config 'default' for dataset alexantonov/chuvash_mono


README.md:   0%|          | 0.00/309 [00:00<?, ?B/s]

Using config 'default' for dataset NeutrinoPit/OpenSubtitles2024-en-ar-batch49


...

Resolving data files:   0%|          | 0/53 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/53 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/53 [00:00<?, ?it/s]

Using config 'default' for dataset hieule/news_corpus_v1_p1


,does_have_urls,id,hf_url
653,False,alexantonov/chuvash_mono,https://huggingface.co/datasets/alexantonov/chuvash_mono
12406,False,NeutrinoPit/OpenSubtitles2024-en-ar-batch49,https://huggingface.co/datasets/NeutrinoPit/OpenSubtitles2024-en-ar-batch49
6715,False,yiyic/oscar_es_train,https://huggingface.co/datasets/yiyic/oscar_es_train
7901,False,gowitheflow/supervised-multilingual,https://huggingface.co/datasets/gowitheflow/supervised-multilingual
1499,False,Babelscape/multinerd,https://huggingface.co/datasets/Babelscape/multinerd
...,...,...,...
6411,False,mteb/eurlex-multilingual,https://huggingface.co/datasets/mteb/eurlex-multilingual
1437,False,FremyCompany/BioLORD-Dataset,https://huggingface.co/datasets/FremyCompany/BioLORD-Dataset
9164,False,KathirKs/CC-MAIN-2017-13_row_wise_20240926_154028,https://huggingface.co/datasets/KathirKs/CC-MAIN-2017-13_row_wise_20240926_154028
10101,False,1231czx/stage2_ep1_6_first8,https://huggingface.co/datasets/1231czx/stage2_ep1_6_first8


In [8]:
ds_df_sample.sort_values(by="does_have_urls").to_csv('sample_with_url_labels.csv',index=False)
ds_df_sample.does_have_urls.value_counts()

,count
does_have_urls,
False,94
True,6


# Check for Multiple Sources